In [4]:
import numpy as np
import ast
import urllib
import matplotlib.pyplot as plt
import random
from collections import defaultdict
import scipy.optimize
from sklearn import linear_model # To perform logistic regression
from sklearn.preprocessing import LabelEncoder # Perform label based encoding here
from sklearn.preprocessing import OneHotEncoder # Perform one hot encoding here
import tensorflow as tf
import array
import gzip
import random
#from tensorflow.keras import Model
from collections import defaultdict

In [5]:
def parseDataFromFile(fname):
    """ Read through the file and return results"""
    for l in open(fname, encoding="utf8"):
        yield ast.literal_eval(l)

In [6]:
dataset = list(parseDataFromFile("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\train.json"))
print(dataset[0])

{'hours': 0.3, 'gameID': 'b96045472', 'hours_transformed': 0.37851162325372983, 'early_access': False, 'date': '2015-04-08', 'text': '+1', 'userID': 'u01561183'}


In [125]:
print(len(dataset))
training_validation_boundary = 165000
# Here, we compute the boundaries between the training and validation set
training_set = dataset[:training_validation_boundary]
validation_set = dataset[training_validation_boundary:]

175000


In [37]:
def perform_duplication(dataset, start, end):
    assert(start <= len(dataset) and end <= len(dataset))
    new_set = []
    print("Start = {}, end = {}".format(start, end))
    for d in dataset[start:end]:
        user, game = d['userID'], d['gameID']
        gamesNotPlayed = []
        for game in gameIDs:
            if(game not in gamesPerUser[user]):
                gamesNotPlayed.append(game)
                
        random.seed()
        randomGame = random.choice(gamesNotPlayed)
        new_data = {'hours': 0, 'gameID': str(randomGame),\
                'hours_transformed': 0, 'early_access': False,\
                'date': '', 'text': '', 'userID': str(userID)}
        new_set.append(new_data)
    return dataset + new_set

In [126]:
usersPerGame = defaultdict(set) # list of userIDs per game
gamesPerUser = defaultdict(set) # list of gameIDs per user
reviewsPerUser = defaultdict(list)
reviewsPerGame = defaultdict(list)
gameIDs = [] # This contains the entire list of games

usersPerGame_training = defaultdict(set) # list of userIDs per game
gamesPerUser_training = defaultdict(set) # list of gameIDs per user
reviewsPerUser_training = defaultdict(list)
reviewsPerGame_training = defaultdict(list)
gameIDs_training = [] # This contains the entire list of games

usersPerGame_validation = defaultdict(set) # list of userIDs per game
gamesPerUser_validation = defaultdict(set) # list of gameIDs per user
reviewsPerUser_validation = defaultdict(list)
reviewsPerGame_validation = defaultdict(list)
gameIDs_validation = [] # This contains the entire list of games

# Computing maps and lists for all samples
for d in dataset:
    userID, gameID = d['userID'], d['gameID']
    usersPerGame[gameID].add(userID)
    gamesPerUser[userID].add(gameID)
    reviewsPerUser[userID].append(d)
    reviewsPerGame[gameID].append(d)
    if gameID not in gameIDs:
        gameIDs.append(gameID)
        
# Computing maps and lists for training set samples
for d in training_set:
    userID, gameID = d['userID'], d['gameID']
    usersPerGame_training[gameID].add(userID)
    gamesPerUser_training[userID].add(gameID) # list of gameIDs per user
    reviewsPerUser_training[userID].append(d)
    reviewsPerGame_training[gameID].append(d)
    if gameID not in gameIDs_training:
        gameIDs_training.append(gameID)
        
# Computing maps and lists for training set samples
for d in validation_set:
    userID, gameID = d['userID'], d['gameID']
    usersPerGame_validation[gameID].add(userID)
    gamesPerUser_validation[userID].add(gameID) # list of gameIDs per user
    reviewsPerUser_validation[userID].append(d)
    reviewsPerGame_validation[gameID].append(d)
    if gameID not in gameIDs_training:
        gameIDs_validation.append(gameID)
print("Done...")

Done...


In [127]:
users = list(gamesPerUser_training.keys())
games = list(usersPerGame_training.keys())
numUsers = len(users)
numGames = len(games)

playPredictionMean = sum([d['hours_transformed'] for d in training_set]) / len(training_set)
print("Average play time is {}".format(playPredictionMean))
alpha = playPredictionMean
userBiases = defaultdict(float)
gameBiases = defaultdict(float)
userGamma = {}
gameGamma = {}
N = 140000

Average play time is 3.7160737367374947


In [128]:
def unpack(theta):
    global alpha
    global userBiases
    global gameBiases
    global userGamma
    global gameGamma
    alpha = theta[0]
    index = 1
    userBiases = dict(zip(users, theta[index:index+numUsers]))
    # Incrementing index by the value of numUsers
    index += numUsers
    gameBiases = dict(zip(games, theta[index:index+numGames]))
    # Incrementing index by the value of numGames
    index += numGames
    for u in users:
        userGamma[u] = theta[index:index+K]
        index += K
    for g in games:
        gameGamma[g] = theta[index:index+K]
        index += K

In [129]:
def inner(x, y):
    return sum([a*b for (a,b) in zip(x, y)])

def prediction(user, game):
    return alpha + userBiases[user] + gameBiases[game] + inner(userGamma[user], gameGamma[game])

def MSE(pred, actual):
    differences = [(x-y)**2 for x, y in zip(pred, actual)]
    return sum(differences)/len(differences)

def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d['userID'], d['gameID']) for d in training_set]
    cost = MSE(predictions, labels)
    print("MSE = {}".format(cost))
    for u in users:
        cost += (lamb/N) * userBiases[u]**2
        for k in range(K):
            cost += (lamb/N) * userGamma[u][k]**2
            
    for g in games:
        cost += (lamb/N) * gameBiases[g]**2
        for k in range(K):
            cost += (lamb/N) * gameGamma[g][k]**2
            
    return cost

In [130]:
def derivative(theta, labels, lamb):
    unpack(theta)
    #N = len(training_set)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dGameBiases = defaultdict(float)
    dUserGamma = {}
    dGameGamma = {}
    for u in reviewsPerUser_training:
        dUserGamma[u] = [0.0 for k in range(K)]
    for g in reviewsPerGame_training:
        dGameGamma[g] = [0.0 for k in range(K)]
    for d in training_set:
        u,g = d['userID'], d['gameID']
        pred = prediction(u, g)
        diff = pred - d['hours_transformed']
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dGameBiases[g] += 2/N*diff
        for k in range(K):
            dUserGamma[u][k] += 2/N*gameGamma[g][k]*diff
            dGameGamma[g][k] += 2/N*userGamma[u][k]*diff
    for u in userBiases:
        dUserBiases[u] += 2 * lamb/N * userBiases[u]
        for k in range(K):
            dUserGamma[u][k] += 2 * lamb/N * userGamma[u][k]
    for g in gameBiases:
        dGameBiases[g] += 2 * lamb/N *gameBiases[g]
        for k in range(K):
            dGameGamma[g][k] += 2 * lamb/N * gameGamma[g][k]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dGameBiases[g] for g in games]
    for u in users:
        dtheta += dUserGamma[u]
    for g in games:
        dtheta += dGameGamma[g]
    return np.array(dtheta)

In [29]:
#K_vals = [5]
lambdas = [10 ** i for i in np.arange(-3, 3, dtype=float)]
for user in reviewsPerUser_training:
    userGamma[user] = [random.random() * 0.1 - 0.05 for k in range(K)]

for game in reviewsPerGame_training:
    gameGamma[game] = [random.random() * 0.1 - 0.05 for k in range(K)]
    
for lamb in lambdas:
    print("Value of K = {}".format(K))
    
    labels = [d['hours_transformed'] for d in training_set]
    x, f, d = scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(numUsers + numGames) + 
                                            [random.random() * 0.1 - 0.05 for k in range(K*(numUsers + numGames))],
                                            derivative, args = (labels, lamb), maxiter = 50, epsilon=1e-4)
    unpack(x)
    predicted_y = [prediction(d['userID'], d['gameID']) for d in validation_set]
    actual_y = [d['hours_transformed'] for d in validation_set]
    mse_new = MSE(predicted_y, actual_y)
    print("MSE on validation set at lambda = {} is {}".format(lamb, mse_new))

Value of K = 5
MSE = 5.691527370000332
MSE = 5.389529713045466
MSE = 5.262000783591928
MSE = 5.2542670069995445
MSE = 5.22587305751773
MSE = 4.938379874795101
MSE = 4.565062472093527
MSE = 4.194528003031602
MSE = 4.023730574266041
MSE = 3.973502293657115
MSE = 3.957380632900566
MSE = 3.9098576842361985
MSE = 3.824551991641621
MSE = 3.682649154212008
MSE = 3.51216552794327
MSE = 3.4227242286335486
MSE = 3.3970663426153456
MSE = 3.3697714406968826
MSE = 3.3288563891263423
MSE = 3.2577367662556407
MSE = 3.2000721085524533
MSE = 3.181449885854326
MSE = 3.1739194897901593
MSE = 3.1592200729593247
MSE = 3.129738558037095
MSE = 3.0896545604186807
MSE = 5.846467874937383
MSE = 3.0872372515366786
MSE = 3.0655535508321425
MSE = 3.0594434296826982
MSE = 3.0562896680617757
MSE = 3.0516135973564023
MSE = 3.0376399993891856
MSE = 3.0152268011034447
MSE = 3.391498629765439
MSE = 3.007948398662462
MSE = 2.9786341327659107
MSE = 2.960473788409955
MSE = 2.9553549549388265
MSE = 2.9517540849232335
MSE = 

MSE = 3.4260134105322972
MSE = 3.4228338601943094
MSE = 3.418214708921065
MSE = 3.412694634087164
MSE = 3.411267155085558
MSE = 3.4107715273134924
MSE = 3.4109794691123887
MSE = 3.4109613983659464
MSE = 3.410277774410846
MSE = 3.40957593237151
MSE = 3.407705452372116
MSE = 3.4069240898735877
MSE = 3.4064162733118057
MSE = 3.4067037344546227
MSE = 3.406322384792796
MSE = 3.4069614550743648
MSE = 3.4075204076784043
MSE = 3.408309690618232
MSE = 3.408735027708896
MSE = 3.4090893983321844
MSE = 3.4133780398455364
MSE = 3.4093096626933606
MSE = 3.4093171410089353
MSE = 3.4094159756353357
MSE = 3.4097985688150105
MSE = 3.409513259577199
MSE = 3.4098119689310464
MSE on validation set at lambda = 100.0 is 3.458801998367456


In [34]:
labels = [d['hours_transformed'] for d in training_set]
x, f, d = scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(numUsers + numGames) + 
                                       [random.random() * 0.1 - 0.05 for k in range(K*(numUsers + numGames))],
                                       derivative, args = (labels, 10), maxiter = 50, epsilon=1e-4)
unpack(x)
predicted_y = [prediction(d['userID'], d['gameID']) for d in validation_set]
actual_y = [d['hours_transformed'] for d in validation_set]
mse_new = MSE(predicted_y, actual_y)
print("After performing gradient descent using the new model, MSE on validation dataset is {}".format(mse_new))

MSE = 5.598071281324904
MSE = 5.447540267356215
MSE = 5.262149029382118
MSE = 5.254506685803133
MSE = 5.225736428129282
MSE = 4.834905333183468
MSE = 4.4446992456349
MSE = 4.110351506954335
MSE = 4.010133745882618
MSE = 3.992534010578454
MSE = 3.9833621130878436
MSE = 3.954393108038159
MSE = 3.891983358790036
MSE = 3.7688294191559932
MSE = 3.611274247714559
MSE = 3.509221124754015
MSE = 3.4799324183262237
MSE = 3.474734079715999
MSE = 3.465368596979112
MSE = 3.4414797674980524
MSE = 3.3857303953044138
MSE = 3.298154240024575
MSE = 3.5804002618974637
MSE = 3.2912893784439086
MSE = 3.226407234478226
MSE = 3.1951605345212233
MSE = 3.1710857225325513
MSE = 3.154641873918803
MSE = 3.1455002862958015
MSE = 3.1078560927727175
MSE = 3.1050877689287883
MSE = 3.0928152956687325
MSE = 3.0676544509442816
MSE = 3.031557201907888
MSE = 2.986160284539363
MSE = 2.941496163975374
MSE = 2.9270472064761726
MSE = 2.9229163124427626
MSE = 2.9196618442283855
MSE = 2.911336326920477
MSE = 2.9037267046184887


In [36]:
#K_vals = [i for i in range(2,10)]
# for K in K_vals:
K = 4
labels = [d['hours_transformed'] for d in training_set]
x, f, d = scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0]*(numUsers + numGames) + 
                                        [random.random() * 0.1 - 0.05 for k in range(K*(numUsers + numGames))],
                                        derivative, args = (labels, 10), maxiter = 50, epsilon=1e-4)
unpack(x)
predicted_y = [prediction(d['userID'], d['gameID']) for d in validation_set]
actual_y = [d['hours_transformed'] for d in validation_set]
mse_new = MSE(predicted_y, actual_y)
print("MSE at K = {} is {}".format(K, mse_new))

MSE = 5.5826643473045925
MSE = 5.4589462415376895
MSE = 5.262141352045876
MSE = 5.254508643258981
MSE = 5.225684356863251
MSE = 4.816379594095981
MSE = 4.425310286022395
MSE = 4.095647709798488
MSE = 3.999875973697815
MSE = 3.9831691861650547
MSE = 3.9721767785967477
MSE = 3.9392707691599043
MSE = 3.8692062370637514
MSE = 3.739307788211196
MSE = 3.5812161806931457
MSE = 3.502713687613748
MSE = 3.4842460459991953
MSE = 3.480655038446243
MSE = 3.4729113004140424
MSE = 3.452432367405462
MSE = 3.402037613808408
MSE = 3.311541801180521
MSE = 6.3802502742782945
MSE = 3.310816900533889
MSE = 3.178577937249094
MSE = 3.3827217802784477
MSE = 3.1246708091770787
MSE = 3.108936349095244
MSE = 3.0958975007166987
MSE = 3.0710191154242756
MSE = 3.038899650716741
MSE = 3.02928844416932
MSE = 3.0168926253841066
MSE = 2.9879311794774694
MSE = 2.95573840750765
MSE = 2.927941056441213
MSE = 2.917222437518171
MSE = 2.942789986649303
MSE = 2.9175823677269626
MSE = 2.91684466599328
MSE = 2.9077097235157274
M

MSE = 2.8745176858994306
MSE = 2.870089692580413
MSE = 2.8711626094011398
MSE = 2.8713687447502405
MSE = 2.869131218393187
MSE = 2.862722600718438
MSE = 2.855581732059783
MSE = 2.845265901074071
MSE = 2.839924590300877
MSE = 2.8311894239393167
MSE = 2.8278296779879124
MSE = 2.818260893956375
MSE at K = 7 is 3.01322710893676
MSE = 5.59944239181365
MSE = 5.446598284210366
MSE = 5.26217490714313
MSE = 5.254531029577915
MSE = 5.225763377722207
MSE = 4.836527245132453
MSE = 4.446395756729901
MSE = 4.111535230347416
MSE = 4.010800536015621
MSE = 3.9930698248276175
MSE = 3.9839985238066866
MSE = 3.955256979349479
MSE = 3.893305528494039
MSE = 3.7705909368765864
MSE = 3.612967606985271
MSE = 3.5089234532539706
MSE = 3.4785351921638274
MSE = 3.473031354263657
MSE = 3.4631573494547796
MSE = 3.4381657662179954
MSE = 3.3807975299351836
MSE = 3.29415481266447
MSE = 3.340820228673227
MSE = 3.2792196194792957
MSE = 3.2142697725991556
MSE = 3.196509347988797
MSE = 3.1863755876829862
MSE = 3.1807493206

In [ ]:
labels = [d['hours_transformed'] for d in training_set]
K = 3
best_models = defaultdict(list)
min_delta = 0
current_mse = 100
last_mse = 200
idx = 0
theta = [alpha] + [0.0] * (numUsers + numGames) + [random.random() * 0.1 - 0.05 for k in range(K * (numUsers + numGames))]
prev_theta = [0] * len(theta)
while last_mse - current_mse > min_delta:
    a, b, c = scipy.optimize.fmin_l_bfgs_b(cost, theta, derivative, args = (labels, 10), maxiter = 5, epsilon=1e-4)
    unpack(a)
    if(idx != 0):
        last_mse = current_mse
        prev_theta = theta
    theta = a
    predicted_y = [prediction(d['userID'], d['gameID']) for d in validation_set]
    actual_y = [d['hours_transformed'] for d in validation_set]
    mse_new = MSE(predicted_y, actual_y)

    current_mse = mse_new
    if(last_mse - current_mse <= min_delta):
        best_models[K] = prev_theta
    idx += 1
    print("MSE at iteration {} is {}".format(idx, mse_new))
    print("Current MSE and Last MSE at iteration {} are {} and {}".format(idx, current_mse, last_mse))
print("At K = {}, value of validation set MSE = {}".format(k, current_mse))

MSE = 5.27807834655407
MSE = 5.1593654901730925
MSE = 4.751238816716987
MSE = 15.445019133227138
MSE = 4.5746471029657485
MSE = 4.072748819315483
MSE = 4.065212497351212
MSE = 4.035739067380541
MSE = 3.928595070587965
MSE = 3.5243833465515726
MSE at iteration 1 is 3.5235074321765505
Current MSE and Last MSE at iteration 1 are 3.5235074321765505 and 200
MSE = 3.5243833465515726
MSE = 3.9181961867168313
MSE = 3.5228249796156432
MSE = 3.520674291410048
MSE = 3.5071353295702563
MSE = 3.484017408414372
MSE = 3.4387806740291658
MSE at iteration 2 is 3.4565097904337105
Current MSE and Last MSE at iteration 2 are 3.4565097904337105 and 3.5235074321765505
MSE = 3.4387806740291658
MSE = 4.124313099423202
MSE = 3.416080433131463
MSE = 3.4145066583616908
MSE = 3.414038434557413
MSE = 3.408600714550458
MSE = 3.398756312213369
MSE at iteration 3 is 3.418448168189802
Current MSE and Last MSE at iteration 3 are 3.418448168189802 and 3.4565097904337105
MSE = 3.398756312213369
MSE = 4.207376260662259
MS

MSE = 3.1183166626761687
MSE = 3.1148908294120052
MSE at iteration 28 is 3.195108910143484
Current MSE and Last MSE at iteration 28 are 3.195108910143484 and 3.200779713156737
MSE = 3.1148908294120052
MSE = 4.0065895533487765
MSE = 3.1132008119845382
MSE = 3.1128043503698466
MSE = 3.112673634186388
MSE = 3.1109395792921752
MSE = 3.1075954161597052
MSE at iteration 29 is 3.1896142607307567
Current MSE and Last MSE at iteration 29 are 3.1896142607307567 and 3.195108910143484
MSE = 3.1075954161597052
MSE = 4.001008022595577
MSE = 3.1059644681772887
MSE = 3.1055797021039186
MSE = 3.1054542888493035
MSE = 3.103760404027207
MSE = 3.1004951373523544
MSE at iteration 30 is 3.1842892231987956
Current MSE and Last MSE at iteration 30 are 3.1842892231987956 and 3.1896142607307567
MSE = 3.1004951373523544
MSE = 3.995586310376935
MSE = 3.098920837335295
MSE = 3.098547380354412
MSE = 3.098427019316591
MSE = 3.096772007866342
MSE = 3.0935829865986566
MSE at iteration 31 is 3.179127515205873
Current M

MSE = 2.927015291432569
MSE = 2.926908672763483
MSE = 2.9258304258408976
MSE = 2.923027770241807
MSE at iteration 54 is 3.061868204302786
Current MSE and Last MSE at iteration 54 are 3.061868204302786 and 3.0649657287210696
MSE = 2.923027770241807
MSE = 3.855933564722357
MSE = 2.9222529258930616
MSE = 2.922068676472071
MSE = 2.9220419667668986
MSE = 2.9219408929404977
MSE = 2.920890796207588
MSE = 2.918181339768135
MSE at iteration 55 is 3.0589272786664026
Current MSE and Last MSE at iteration 55 are 3.0589272786664026 and 3.061868204302786
MSE = 2.918181339768135
MSE = 3.852741254300596
MSE = 2.917446383519737
MSE = 2.9172701660034703
MSE = 2.9172448593743874
MSE = 2.917148892276131
MSE = 2.916125503825045
MSE = 2.913502480559544
MSE at iteration 56 is 3.0561359752542865
Current MSE and Last MSE at iteration 56 are 3.0561359752542865 and 3.0589272786664026
MSE = 2.913502480559544
MSE = 3.849592004362899
MSE = 2.912803341718058
MSE = 2.912634431527977
MSE = 2.9126104036637672
MSE = 2.9

MSE = 2.811725498246527
MSE at iteration 79 is 3.0255227006161176
Current MSE and Last MSE at iteration 79 are 3.0255227006161176 and 3.0258466986998656
MSE = 2.811725498246527
MSE = 3.7285540207257473
MSE = 2.8104949181624796
MSE = 2.8102282474860325
MSE = 2.8101934217322446
MSE = 2.810066004659813
MSE = 2.8089824755655766
MSE = 2.805971449105503
MSE at iteration 80 is 3.0252452042838756
Current MSE and Last MSE at iteration 80 are 3.0252452042838756 and 3.0255227006161176
MSE = 2.805971449105503
MSE = 3.7220295247463078
MSE = 2.8047168824044095
MSE = 2.8044459596469222
MSE = 2.8044102490828635
MSE = 2.8042797532571515
MSE = 2.8031837830882544
MSE = 2.800133732262888
MSE at iteration 81 is 3.0250117864661865
Current MSE and Last MSE at iteration 81 are 3.0250117864661865 and 3.0252452042838756
MSE = 2.800133732262888
MSE = 3.7157337451506556
MSE = 2.798864992085799
MSE = 2.7985915256554876
MSE = 2.798555204986096
MSE = 2.798422570388552
MSE = 2.7973179669684654
MSE = 2.794243217771364

In [45]:
print(len(best_models))
for key, value in best_models.items():
    unpack(value)
    predicted_y = [prediction(d['userID'], d['gameID']) for d in validation_set]
    actual_y = [d['hours_transformed'] for d in validation_set]
    mse_new = MSE(predicted_y, actual_y)
    print("MSE for model is {} at K = {}".format(mse_new, key))

2
MSE for model is 3.0147011517944424 at K = 2
MSE for model is 3.0067143778065217 at K = 3


In [120]:
unpack(best_models[3])
predictions = open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\predictions_Hours_gammav3.txt", 'w')
for l in open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\pairs_Hours.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,g = l.strip().split('-')
    pred = prediction(u, g)
    predictions.write(u + '-' + g + ',' + str(pred) + '\n')

predictions.close()

IndexError: list index out of range

In [98]:
dataset = list(parseDataFromFile("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\train.json"))

In [102]:
#random.shuffle(dataset)
def kFoldCrossValidation(dataset, K):
    lengthGroup = len(dataset) // K
    idx = 0
    train, test = defaultdict(), defaultdict()
    
    while(idx < K):
        test_set = dataset[idx*lengthGroup : (idx+1) * lengthGroup]
        if(idx == 0):
            train_set = dataset[(idx+1) * lengthGroup:]
        elif(idx > 1 and idx < K - 1):
            train_set = dataset[:idx * lengthGroup] + dataset[(idx+1)*lengthGroup:]
        elif(idx == K-1):
            train_set = dataset[:(idx * lengthGroup)]
        idx += 1
        train[idx] = train_set
        test[idx] = test_set
    
    return train, test
L = 5

K_vals = [i for i in np.arange(1, 7)]
#print(K_vals)
best_models = defaultdict(list) # Best models for each value of K
for K in K_vals:
    theta = [alpha] + [0.0] * (numUsers + numGames) + [random.random() * 0.1 - 0.05 for k in range(K * (numUsers + numGames))]
    prev_theta = [0] * len(theta)
    train_dataset, test_dataset = kFoldCrossValidation(dataset, L)
    for i in range(L):
        train_set = train_dataset[i+1]
        test_set = test_dataset[i+1]
        print("Train and test lengths are {}, {}".format(len(train_set), len(test_set)))
        labels = [d['hours_transformed'] for d in train_set]
        print("Length of labels = {}".format(len(labels)))
        min_delta = 0.0001 # Changing the values of the MSE here
        current_mse = 100
        last_mse = 200
        idx = 0
        while (last_mse - current_mse) > min_delta:
            a, b, c = scipy.optimize.fmin_l_bfgs_b(cost, theta, derivative, args = (labels, 10),maxiter=5, epsilon=1e-4)
            unpack(a)
            if(idx != 0):
                last_mse = current_mse
                prev_theta = theta
            theta = a
            predicted_y = [prediction(d['userID'], d['gameID']) for d in test_set]
            actual_y = [d['hours_transformed'] for d in test_set]
            mse_new = MSE(predicted_y, actual_y)
            current_mse = mse_new
            if(last_mse - current_mse <= min_delta):
                best_models[k] = prev_theta
            idx += 1
            print("MSE at iteration {} is {}".format(idx, mse_new))
            print("Current MSE and Last MSE at iteration {} are {} and {}".format(idx, current_mse, last_mse))
        #print("For the idx = {}, validation set MSE = {}".format(idx, mse_new))
        print("At i = {}, the MSE after K Cross validation is {}".format(i, mse_new))
    print("At K = {}, value of validation set MSE = {}".format(K, current_mse))

Train and test lengths are 140000, 35000
Length of labels = 140000
MSE = 5.618534660269767
MSE = 5.466447409814028
MSE = 5.289154040537968
MSE = 5.28922909724163
MSE = 5.289154721896101
MSE = 5.289153509506456
MSE = 5.289153745150704
MSE = 5.289153376616874
MSE = 5.289153473274315
MSE = 5.2891534026406735
MSE = 5.471123439274201
MSE = 5.2958977680216
MSE = 5.289469108553169
MSE = 5.289173174016408
MSE = 5.2891555120279605
MSE = 5.289153756081419
MSE = 5.289153473122188
MSE = 5.289153417325973
MSE = 5.289153405730066
MSE = 5.289153403291908
MSE = 5.2891534027779725
MSE = 5.28915340266963
MSE = 5.289153402646716
MSE = 5.289153402641944
MSE = 5.289153402640957
MSE = 5.289153402640776
MSE = 5.2891534026406175
MSE = 5.289153402640664
MSE = 5.289153402640613
MSE = 5.289153402640618
MSE = 5.289422765987009
MSE = 5.289166973085123
MSE = 5.28915436285245
MSE = 5.289153520687818
MSE = 5.289153423791983
MSE = 5.2891534069384205
MSE = 5.289153403540562
MSE = 5.289153402830204
MSE = 5.2891534026806

MSE = 4.917369620225
MSE = 4.9199674069925665
MSE = 4.917465713774218
MSE = 4.917374352083287
MSE = 4.917370007751612
MSE = 4.917369675524575
MSE = 4.91736963037191
MSE = 4.9173696222115595
MSE = 4.917369620619364
MSE = 4.917369620303498
MSE = 4.917369620240743
MSE = 4.917369620228092
MSE = 4.91736962022552
MSE = 4.917369620225119
MSE = 4.917369620225107
MSE = 4.917369620224982
MSE = 4.917369620225
MSE = 4.917369620224987
MSE = 4.917369620225015
MSE = 4.917369620225016
MSE = 4.917369620224989
MSE = 4.91736489598565
MSE = 4.916855306764488
MSE = 4.916997170320577
MSE = 4.916698252399488
MSE at iteration 9 is 3.909175164397703
Current MSE and Last MSE at iteration 9 are 3.909175164397703 and 3.9231830982193685
MSE = 4.916698252399488
MSE = 4.9225995175621104
MSE = 4.9158166623026665
MSE = 4.940397400782235
MSE = 4.914565592072497
MSE = 4.915274497328356
MSE = 4.914589085411653
MSE = 4.915056616961703
MSE = 4.914689533022489
MSE = 4.9149596529339235
MSE = 4.914765562512668
MSE = 4.9149134

MSE = 4.073679364043515
MSE = 4.073853697643825
MSE = 4.073732244847299
MSE = 4.073825671242164
MSE = 4.073762287727719
MSE = 4.073811704497125
MSE = 4.073778623106571
MSE = 4.073804590330723
MSE = 4.073811704497125
MSE at iteration 5 is 3.176143117210747
Current MSE and Last MSE at iteration 5 are 3.176143117210747 and 3.1781657421260014
MSE = 4.073811704497125
MSE = 4.760478479866243
MSE = 4.073699193843883
MSE = 4.073552450759479
MSE = 4.073336458717318
MSE = 4.073073615394002
MSE = 4.073227354915391
MSE = 4.07307902314082
MSE = 4.073184139646114
MSE = 4.073218120128375
MSE = 4.073227354915391
MSE = 4.072866387714621
MSE = 4.072847779233357
MSE = 4.073131071908893
MSE = 4.072933515550868
MSE = 4.073085202753474
MSE = 4.072982245862258
MSE = 4.0730623991223265
MSE = 4.073008729901665
MSE = 4.073050804063573
MSE = 4.073022827058454
MSE = 4.073044838045066
MSE = 4.073050804063573
MSE at iteration 6 is 3.173859007528121
Current MSE and Last MSE at iteration 6 are 3.173859007528121 and 3

MSE = 4.065692470791359
MSE = 4.065464053002686
MSE = 4.065603955782276
MSE = 4.06567168769806
MSE = 4.0656254980049145
MSE = 4.065661427619287
MSE = 4.065637311662228
MSE = 4.065656225601791
MSE = 4.065660324255632
MSE = 4.065661427619287
MSE = 4.065227067742616
MSE = 4.065307453272839
MSE = 4.06557160938942
MSE = 4.065388079586251
MSE = 4.065528856978631
MSE = 4.065433487113396
MSE = 4.065507626688194
MSE = 4.065458033743489
MSE = 4.065441756576145
MSE = 4.065458033743489
MSE at iteration 17 is 3.1517613079388638
Current MSE and Last MSE at iteration 17 are 3.1517613079388638 and 3.1535735914724325
MSE = 4.065458033743489
MSE = 4.763120424162298
MSE = 4.065357399474911
MSE = 4.065234222443659
MSE = 4.0650591761033645
MSE = 4.064820059979262
MSE = 4.064967689807486
MSE = 4.065037754246253
MSE = 4.064989995803826
MSE = 4.065027167631278
MSE = 4.065002223569604
MSE = 4.065021796199409
MSE = 4.065026030026927
MSE = 4.065027167631278
MSE = 4.064560284214697
MSE = 4.06465802290407
MSE = 4.

MSE = 4.057161597656195
MSE = 4.8183722356802985
MSE = 4.057028506860402
MSE = 4.0569176652616665
MSE = 4.056792541223891
MSE = 4.056561206209415
MSE = 4.056710895728824
MSE = 4.056583949956297
MSE = 4.056677326468574
MSE = 4.056703755322687
MSE = 4.056710895728824
MSE = 4.056216932270868
MSE = 4.056438255436805
MSE = 4.05664352079989
MSE = 4.056502068815758
MSE = 4.056611090099053
MSE = 4.056537533368564
MSE = 4.056594881233952
MSE = 4.056556599531536
MSE = 4.056586619237694
MSE = 4.0565666893999355
MSE = 4.056582364501727
MSE = 4.056571987300384
MSE = 4.056582364501727
MSE at iteration 30 is 3.127312158270252
Current MSE and Last MSE at iteration 30 are 3.127312158270252 and 3.128796213010848
MSE = 4.056582364501727
MSE = 4.757040699558643
MSE = 4.056487269176532
MSE = 4.056378346719927
MSE = 4.056215042969538
MSE = 4.055945801448581
MSE = 4.056122150175971
MSE = 4.0559741074899085
MSE = 4.056083573143264
MSE = 4.056114012475964
MSE = 4.056122150175971
MSE = 4.055522876096877
MSE = 4

...

MSE = 2.5963587219817965
MSE = 2.59629726457039
MSE = 2.595455825906242
MSE = 2.59342144745708
MSE at iteration 82 is 2.7013884838300597
Current MSE and Last MSE at iteration 82 are 2.7013884838300597 and 2.703560148852317
MSE = 2.59342144745708
MSE = 3.528801027138496
MSE = 2.592642072096666
MSE = 2.5925108949086564
MSE = 2.5924951255973667
MSE = 2.5924352246351585
MSE = 2.5916024050174116
MSE = 2.5895950455685193
MSE at iteration 83 is 2.6992809310479444
Current MSE and Last MSE at iteration 83 are 2.6992809310479444 and 2.7013884838300597
MSE = 2.5895950455685193
MSE = 3.525651957013077
MSE = 2.5888313329012456
MSE = 2.588703300252805
MSE = 2.5886879508778415
MSE = 2.5886295827299945
MSE = 2.5878053915857193
MSE = 2.585824831303991
MSE at iteration 84 is 2.6972356630634997
Current MSE and Last MSE at iteration 84 are 2.6972356630634997 and 2.6992809310479444
MSE = 2.585824831303991
MSE = 3.5225513063206217
MSE = 2.5850764151228196
MSE = 2.5849514568750704
MSE = 2.584936518677689
MSE

MSE at iteration 107 is 2.6628739276578246
Current MSE and Last MSE at iteration 107 are 2.6628739276578246 and 2.664017752293306
MSE = 2.51201428648461
MSE = 3.4608246762750565
MSE = 2.51151212816056
MSE = 2.51143624470044
MSE = 2.511427775583535
MSE = 2.5113949635103694
MSE = 2.5107388034427958
MSE = 2.5092447438875816
MSE at iteration 108 is 2.661748677975219
Current MSE and Last MSE at iteration 108 are 2.661748677975219 and 2.6628739276578246
MSE = 2.5092447438875816
MSE = 3.4584402550418947
MSE = 2.508749547106716
MSE = 2.508675021063561
MSE = 2.5086667227099473
MSE = 2.5086345551692215
MSE = 2.5079832340154478
MSE = 2.5065024317637916
MSE at iteration 109 is 2.6606406225212593
Current MSE and Last MSE at iteration 109 are 2.6606406225212593 and 2.661748677975219
MSE = 2.5065024317637916
MSE = 3.4560730163321973
MSE = 2.5060139619661395
MSE = 2.5059407469878856
MSE = 2.5059326128065478
MSE = 2.50590106540498
MSE = 2.5052544238199808
MSE = 2.503786421572
MSE at iteration 110 is 2.

MSE = 2.4370106940957768
MSE = 2.4369817413269037
MSE = 2.4369771708335914
MSE = 2.4369593358657404
MSE = 2.4365614896868757
MSE = 2.4356508217090655
MSE at iteration 132 is 2.6323502835571957
Current MSE and Last MSE at iteration 132 are 2.6323502835571957 and 2.633033391843059
MSE = 2.4356508217090655
MSE = 3.398422086711786
MSE = 2.435388177784364
MSE = 2.4353572772067276
MSE = 2.4353527966341577
MSE = 2.4353352763700307
MSE = 2.435271627747938
MSE = 2.4349128401323252
MSE = 2.4332054554779865
MSE at iteration 133 is 2.6313112971928327
Current MSE and Last MSE at iteration 133 are 2.6313112971928327 and 2.6323502835571957
MSE = 2.4332054554779865
MSE = 3.389473646314259
MSE = 2.4328062782518804
MSE = 2.432752911060147
MSE = 2.4327485172455967
MSE = 2.4327311801052827
MSE = 2.4326656414405106
MSE = 2.432065123752081
MSE = 2.4301676008326236
MSE at iteration 134 is 2.6299845259504706
Current MSE and Last MSE at iteration 134 are 2.6299845259504706 and 2.6313112971928327
MSE = 2.430167

MSE = 4.937963058681599
MSE = 5.668692182183066
MSE = 4.934593121201114
MSE = 4.9335270798206015
MSE = 4.932843608249455
MSE = 4.9355505100110735
MSE = 4.932801477209152
MSE = 4.9328106988895195
MSE = 4.932835541358757
MSE = 4.93281825453878
MSE = 4.932813164834902
MSE = 4.932817151528161
MSE = 4.93281825453878
MSE = 4.949557405606684
MSE = 4.932214444275723
MSE = 4.932509014160109
MSE = 4.932176702419185
MSE = 4.932185671311277
MSE at iteration 6 is 3.989813026107253
Current MSE and Last MSE at iteration 6 are 3.989813026107253 and 4.018050495433232
MSE = 4.932185671311277
MSE = 5.660392674207512
MSE = 4.929320906402009
MSE = 4.92833005060496
MSE = 4.9277756243083655
MSE = 4.930850790938085
MSE = 4.927813480744909
MSE = 4.927761432288211
MSE = 4.927771770190392
MSE = 4.927774787210579
MSE = 4.927775449293146
MSE = 4.927775587994731
MSE = 4.927775616785539
MSE = 4.927775622750461
MSE = 4.927775623985702
MSE = 4.927775624241431
MSE = 4.927775624294559
MSE = 4.927775624305454
MSE = 4.927

KeyboardInterrupt: 

In [101]:
unpack(a)
predictions = open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\predictions_Hours_gammav4.txt", 'w')
for l in open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\pairs_Hours.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,g = l.strip().split('-')
    pred = prediction(u, g)
    predictions.write(u + '-' + g + ',' + str(pred) + '\n')

predictions.close()

In [111]:
import numpy as np

dataset = list(parseDataFromFile("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\train.json"))
print(len(dataset))
training_validation_boundary = 140000
# Here, we compute the boundaries between the training and validation set
training_dataset = dataset[:training_validation_boundary]
validation_dataset = dataset[training_validation_boundary:]

usersPerGame = defaultdict(set) # list of userIDs per game
gamesPerUser = defaultdict(set) # list of gameIDs per user
reviewsPerUser = defaultdict(list)
reviewsPerGame = defaultdict(list)
gameIDs = [] # This contains the entire list of games

#dataset = training_dataset

for d in dataset:
    user, game = d['userID'], d['gameID']
    usersPerGame[game].add(user)
    gamesPerUser[user].add(game)
    reviewsPerUser[user].append(d)
    reviewsPerGame[game].append(d)
    # Inserting the gameID into the list of games
    gameIDs.append(game)

users = list(gamesPerUser.keys())
games = list(usersPerGame.keys())
numUsers = len(users)
numGames = len(games)

playPredictionMean = sum([d['hours_transformed'] for d in training_dataset]) / len(training_dataset)

alpha = playPredictionMean
userBiases = defaultdict(float)
gameBiases = defaultdict(float)
N = len(training_dataset)

# Compute the prediction for the user and game
def prediction(user, game):
    return alpha + userBiases[user] + gameBiases[game]

def unpack(theta):
    """ theta = [alpha, beta_u, beta_i]"""
    global alpha
    global userBiases
    global gameBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:numUsers+1]))
    gameBiases = dict(zip(games, theta[1+numUsers:]))

def MSE(predictions, labels):
    differences = [(x-y)**2 for x, y in zip(predictions, labels)]
    return sum(differences)/len(differences)
    
def cost(theta, labels, lamb):
    unpack(theta) 
    predictions = [prediction(d['userID'], d['gameID']) for d in training_dataset]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb/N * (userBiases[u] ** 2)
    for g in gameBiases:
        cost += lamb/N * (gameBiases[g] ** 2)
    return cost

def derivative(theta, labels, lamb):
    # Current values of theta are unpacked
    unpack(theta)
    # Compute deriv_alpha, deriv_beta_g and deriv_beta_u
    dalpha = 0
    dUserBiases = defaultdict(float)
    dGameBiases = defaultdict(float)
    # For sample in dataset
    for d in training_dataset:
        user, game = d['userID'], d['gameID']
        pred = prediction(user, game)
        diff = pred - d['hours_transformed']
        dalpha += 2/N * diff
        dUserBiases[user] += 2/N * diff
        dGameBiases[game] += 2/N * diff
    for u in userBiases:
        dUserBiases[u] += 2 * lamb/N * userBiases[u]
    for g in gameBiases:
        dGameBiases[g] += 2 * lamb/N * gameBiases[g]
        
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dGameBiases[g] for g in games]
    return np.array(dtheta)

print("Prediction mean for validation set is {}".format(playPredictionMean))
# Compute the play prediction mean
alwaysPredictMean = [playPredictionMean for d in validation_dataset]
labels = [d['hours_transformed'] for d in validation_dataset]
mse = MSE(alwaysPredictMean, labels)
print("Always predict mean MSE is {}".format(mse))

K = 5
train_dataset, test_dataset = kFoldCrossValidation(dataset, K)
best_models = defaultdict(list)
lambdas = [10 ** i for i in range(-3, 3)]
for lamb in lambdas:
    for i in range(K):
        train_set = train_dataset[i+1]
        test_set = test_dataset[i+1]
        current_mse = 200
        last_mse = 0
        delta = 0
        while (current_mse - last_mse > delta):
            print("Lengths of train and test set are {} and {}".format(len(train_set), len(test_set)))
            labels = [d['hours_transformed'] for d in train_set]
            x, f, d = scipy.optimize.fmin_l_bfgs_b(cost, [playPredictionMean] + [0.0]*(numUsers + numGames), derivative,\
                                               args = (labels, lamb), maxiter = 10, iprint=100, epsilon=1e-4)
            y_act = [prediction(d['userID'], d['gameID']) for d in test_set]
            y_pred = [d['hours_transformed'] for d in test_set]
            mse_val = MSE(y_act, y_pred)
            if(idx != 0):
                last_mse = current_mse
            current_mse = mse_new
            idx += 1
            print("At iteration {}, current and last MSE are {} and {}".format(idx, current_mse, last_mse))
    best_models[lamb] = x
    print("Optimization done and model saved for lamb = {}".format(lamb))
print("Optimizer done...")

175000
Prediction mean for validation set is 3.7184092445205623
Always predict mean MSE is 5.25142613259121
Lengths of train and test set are 140000 and 35000
MSE = 5.289168784082523
MSE = 5.292016332325914
MSE = 5.289207998281072
MSE = 5.289152503691967
MSE = 5.289168602284352
MSE = 5.289149263333331
MSE = 5.2891564671976745
MSE = 5.289148986876805
MSE = 5.289152417311495
MSE = 5.289149224148325
MSE = 5.289148986876805
MSE = 19.93820642249423
MSE = 5.387846691002906
MSE = 5.2926081980043325
MSE = 5.289324641072224
MSE = 5.289163080811308
MSE = 5.289150954062757
MSE = 5.289149356800639
MSE = 5.289149062787302
MSE = 5.289149002768753
MSE = 5.289148990218054
MSE = 5.289148987579894
MSE = 5.289148987024729
MSE = 5.289148986907804
MSE = 5.289148986883304
MSE = 5.289148986878206
MSE = 5.2891489868771675
MSE = 5.2891489868767945
MSE = 5.2891489868768655
MSE = 5.289148986876847
MSE = 5.289148986876802
MSE = 5.2892056893242225
MSE = 5.289153113930836
MSE = 5.289149510059846
MSE = 5.28914908183

MSE = 5.00885268006881
MSE = 5.00885268005829
MSE = 5.008852680056147
MSE = 5.008852680055709
MSE = 5.008852680055685
MSE = 5.008852680055674
MSE = 5.008852680055664
MSE = 5.008147772311984
MSE = 4.987592085723148
MSE = 4.986918947480436
MSE = 4.986022229872944
MSE = 4.98516349042684
MSE = 5.006271779321484
MSE = 4.983795848915842
MSE = 4.99372926341629
MSE = 4.9842405985573714
MSE = 5.078877677404176
MSE = 4.981783242294604
MSE = 5.175230522523789
MSE = 4.981660750286672
At iteration 25, current and last MSE are 3.888391224893484 and 200
Lengths of train and test set are 140000 and 35000
MSE = 5.266707210059231
MSE = 5.178212821886435
MSE = 4.891514496766213
MSE = 16.64106091295773
MSE = 4.7957570702070464
MSE = 4.668308672385294
MSE = 4.652074824553146
MSE = 4.591321853521145
MSE = 4.314221857515991
MSE = 4.216820851926769
MSE = 4.169798660633075
MSE = 4.16854066735375
MSE = 4.102300810406715
MSE = 4.051517116718102
At iteration 26, current and last MSE are 3.888391224893484 and 200


MSE = 5.289150950382504
MSE = 5.289149356129533
MSE = 5.289149062608046
MSE = 5.289149002687431
MSE = 5.2891489901574715
MSE = 5.289148987523666
MSE = 5.289148986969458
MSE = 5.289148986852805
MSE = 5.2891489868282955
MSE = 5.289148986823156
MSE = 5.2891489868221555
MSE = 5.289148986821872
MSE = 5.289148986821833
MSE = 5.28914898682185
MSE = 5.289148986821836
MSE = 5.289205685606614
MSE = 5.289153113156802
MSE = 5.289149509863235
MSE = 5.2891490817478
MSE = 5.289149006172954
MSE = 5.289148990876673
MSE = 5.289148987676515
MSE = 5.289148987002231
MSE = 5.28914898685976
MSE = 5.289148986829798
MSE = 5.289148986823386
MSE = 5.2891489868221075
MSE = 5.2891489868218065
MSE = 5.289148986821741
MSE = 5.289148986821764
MSE = 5.289148986821813
MSE = 5.289148986821706
MSE = 5.289148986821843
MSE = 5.289148986821686
MSE = 5.289148986821739
At iteration 34, current and last MSE are 3.888391224893484 and 200
Lengths of train and test set are 140000 and 35000
MSE = 5.287648598896488
MSE = 5.22923311

MSE = 5.289149280771324
MSE = 5.289156330938585
MSE = 5.289148981734162
MSE = 5.289149137996481
MSE = 5.289149280771324
MSE = 18.739002282628118
MSE = 5.364042711091883
MSE = 5.291733201674896
MSE = 5.28928153141197
MSE = 5.28916017340375
MSE = 5.289150834472588
MSE = 5.28914957493657
MSE = 5.289149341171165
MSE = 5.289149293403591
MSE = 5.289149283423807
MSE = 5.289149281328752
MSE = 5.289149280888502
MSE = 5.289149280795997
MSE = 5.2891492807765745
MSE = 5.289149280772401
MSE = 5.289149280771532
MSE = 5.289149280771366
MSE = 5.2891492807713965
MSE = 5.2891492807713725
MSE = 5.289149280771326
MSE = 5.289195250947068
MSE = 5.289151187097878
MSE = 5.289149337500323
MSE = 5.289149277356406
MSE = 5.2891492793646435
MSE = 5.289149280443705
MSE = 5.289149280700854
MSE = 5.289149280756416
MSE = 5.2891492807683385
MSE = 5.2891492807706335
MSE = 5.28914928077128
MSE = 5.289149280771387
MSE = 5.289149280771429
MSE = 5.289149280771375
MSE = 5.289149280771332
MSE = 5.28914928077132
MSE = 5.289149

In [105]:
unpack(x)
#new_theta = [alpha] + [0.0]*(numUsers + numGames)
predictions = open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\predictions_Hours_22.txt", 'w')
for l in open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\pairs_Hours.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,g = l.strip().split('-')
    pred = prediction(u, g)
    predictions.write(u + '-' + g + ',' + str(pred) + '\n')

predictions.close()

print("User name used for submission is Severus Snape")

User name used for submission is Severus Snape


In [116]:
import numpy as np

usersPerGame = defaultdict(set) # list of userIDs per game
gamesPerUser = defaultdict(set) # list of gameIDs per user
reviewsPerUser = defaultdict(list)
reviewsPerGame = defaultdict(list)
gameIDs = [] # This contains the entire list of games

#dataset = training_dataset

dataset = list(parseDataFromFile("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\train.json"))
N = len(training_dataset)
for d in dataset:
    user, game = d['userID'], d['gameID']
    usersPerGame[game].add(user)
    gamesPerUser[user].add(game)
    reviewsPerUser[user].append(d)
    reviewsPerGame[game].append(d)
    # Inserting the gameID into the list of games
    gameIDs.append(game)

users = list(gamesPerUser.keys())
games = list(usersPerGame.keys())
numUsers = len(users)
numGames = len(games)

playPredictionMean = sum([d['hours_transformed'] for d in training_dataset]) / len(training_dataset)

alpha = playPredictionMean
userBiases = defaultdict(float)
gameBiases = defaultdict(float)
N = len(training_dataset)

# Compute the prediction for the user and game
def prediction(user, game):
    return alpha + userBiases[user] + gameBiases[game]

def unpack(theta):
    """ theta = [alpha, beta_u, beta_i]"""
    global alpha
    global userBiases
    global gameBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:numUsers+1]))
    gameBiases = dict(zip(games, theta[1+numUsers:]))

def MSE(predictions, labels):
    differences = [(x-y)**2 for x, y in zip(predictions, labels)]
    return sum(differences)/len(differences)
    
def cost(theta, labels, lamb):
    unpack(theta) 
    predictions = [prediction(d['userID'], d['gameID']) for d in dataset]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in userBiases:
        cost += lamb/N * (userBiases[u] ** 2)
    for g in gameBiases:
        cost += lamb/N * (gameBiases[g] ** 2)
    return cost

def derivative(theta, labels, lamb):
    # Current values of theta are unpacked
    unpack(theta)
    # Compute deriv_alpha, deriv_beta_g and deriv_beta_u
    dalpha = 0
    dUserBiases = defaultdict(float)
    dGameBiases = defaultdict(float)
    # For sample in dataset
    for d in training_dataset:
        user, game = d['userID'], d['gameID']
        pred = prediction(user, game)
        diff = pred - d['hours_transformed']
        dalpha += 2/N * diff
        dUserBiases[user] += 2/N * diff
        dGameBiases[game] += 2/N * diff
    for u in userBiases:
        dUserBiases[u] += 2 * lamb/N * userBiases[u]
    for g in gameBiases:
        dGameBiases[g] += 2 * lamb/N * gameBiases[g]
        
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dGameBiases[g] for g in games]
    return np.array(dtheta)

print("Prediction mean for validation set is {}".format(playPredictionMean))
# Compute the play prediction mean
alwaysPredictMean = [playPredictionMean for d in validation_dataset]
labels = [d['hours_transformed'] for d in validation_dataset]
mse = MSE(alwaysPredictMean, labels)
print("Always predict mean MSE is {}".format(mse))
labels = [d['hours_transformed'] for d in training_dataset]
MSE_vals = {}
lambdas = [10 ** i for i in range(-5, 5)]
for lamb in lambdas:
    x, f, d = scipy.optimize.fmin_l_bfgs_b(cost, [playPredictionMean] + [0.0]*(numUsers + numGames), derivative,\
                                           args = (labels, lamb), iprint=100, epsilon=1e-4)
    unpack(x)
    predicted_y = [prediction(d['userID'], d['gameID']) for d in validation_dataset]
    actual_y = [d['hours_transformed'] for d in validation_dataset]
    mse_new = MSE(predicted_y, actual_y)
    print("Lambda = {}, validation set MSE = {}".format(lamb, mse_new))
    MSE_vals[lamb] = mse_new
print("Optimizer done...")

Prediction mean for validation set is 3.7184092445205623
Always predict mean MSE is 5.25142613259121
MSE = 5.287433813783579
MSE = 5.168004738206718
MSE = 4.757567663470286
MSE = 15.555479926389705
MSE = 4.562552841443187
MSE = 4.077161613677074
MSE = 4.063715454032573
MSE = 4.0117954300323895
MSE = 3.833949167275927
MSE = 3.491372155392575
MSE = 3.305164433755272
MSE = 3.095724120518387
MSE = 3.004129309932686
MSE = 2.9053451943973587
MSE = 2.851854315198037
MSE = 2.809474212450705
MSE = 2.786779224914513
MSE = 2.7724765491513312
MSE = 5.726788240716089
MSE = 2.7723508447502168
MSE = 2.7646157736034653
MSE = 2.758145113978325
MSE = 2.7538017244473147
MSE = 2.749579360967079
MSE = 2.7444980299196224
MSE = 2.7392426926650635
MSE = 2.741114076777445
MSE = 2.73740730121326
MSE = 2.735267030963782
MSE = 2.734333907125631
MSE = 2.7335469010311533
MSE = 2.7329082582409363
MSE = 2.767014165277098
MSE = 2.732893057735649
MSE = 2.7327693723012985
MSE = 2.732480116976254
MSE = 2.732286688215395


MSE = 2.7310450770357475
MSE = 2.7310448676597154
MSE = 2.7310437286331957
MSE = 2.731043540348543
MSE = 2.731043370806042
MSE = 2.731042611582307
MSE = 2.7310401090709906
MSE = 2.7310393538232587
MSE = 2.7310376216468764
MSE = 2.7310359394672252
MSE = 2.731034997180109
MSE = 2.7310344235174266
MSE = 2.731186321136854
MSE = 2.7310343753808275
MSE = 2.7310342503428116
MSE = 2.7310366062310742
MSE = 2.7310341112763212
MSE = 2.7310338442817335
MSE = 2.731032751459121
MSE = 2.731031906770736
MSE = 2.7310309905545216
MSE = 2.731028989753876
MSE = 2.731109553180263
MSE = 2.7310289085608455
MSE = 2.7310283674050635
MSE = 2.7310277658247366
MSE = 2.731027470620106
MSE = 2.7310268197534366
MSE = 2.731026409691574
MSE = 2.7310262228462534
MSE = 2.7310849571822775
MSE = 2.731026210017191
MSE = 2.7310261726345475
MSE = 2.731026110466129
MSE = 2.731026008479234
MSE = 2.7310258196844184
MSE = 2.7310256056821594
MSE = 2.7310257984686657
MSE = 2.7310254921434467
MSE = 2.7310255900475937
MSE = 2.731025

MSE = 2.731080255266524
MSE = 2.7310758182477906
MSE = 2.7310700972083426
MSE = 2.731066722303714
MSE = 2.731070918800227
MSE = 2.731065448361693
MSE = 2.731064213898614
MSE = 2.7310629854477284
MSE = 2.731061832259103
MSE = 2.731064949532641
MSE = 2.73106085745229
MSE = 2.731059030343304
MSE = 2.731057195840446
MSE = 2.7310560280662353
MSE = 2.7310532232560973
MSE = 2.731049131766624
MSE = 2.731065385733036
MSE = 2.7310486854253986
MSE = 2.731047555710557
MSE = 2.731047514410544
MSE = 2.7310473075808526
MSE = 2.731047237902309
MSE = 2.7310458815162484
MSE = 2.731042294595486
MSE = 2.7310544660856078
MSE = 2.7310420157390203
MSE = 2.7310410275217794
MSE = 2.7310396125203313
MSE = 2.7310386469383183
MSE = 2.731036927121933
MSE = 2.731035438825875
MSE = 2.7310467101237683
MSE = 2.7310351856892985
MSE = 2.7310346597224444
MSE = 2.731034495461962
MSE = 2.731034302700768
MSE = 2.731034072302362
MSE = 2.731033524819845
MSE = 2.7310333782725276
MSE = 2.731033162597251
MSE = 2.731032627066967


Lambda = 0.1, validation set MSE = 3.0682149577653166
MSE = 5.287433813783579
MSE = 5.168004738206718
MSE = 4.757567663470286
MSE = 15.550578133952362
MSE = 4.5632273376613295
MSE = 4.077157089054892
MSE = 4.063720120930918
MSE = 4.01183524747517
MSE = 3.834103738301693
MSE = 3.4904003856954646
MSE = 3.3050471809464566
MSE = 3.0941463628817254
MSE = 3.00240688478539
MSE = 2.904338322791949
MSE = 2.848798709353993
MSE = 2.807431289276749
MSE = 2.7873744274092784
MSE = 2.7734943493691704
MSE = 4.671686199434022
MSE = 2.77333306301027
MSE = 2.765747990280907
MSE = 2.7577821373700124
MSE = 2.7542142574100863
MSE = 2.7513212083454994
MSE = 2.746683801410092
MSE = 2.742030193981398
MSE = 2.7420759212771935
MSE = 2.7400443999726605
MSE = 2.7380743514824086
MSE = 2.7372353862737935
MSE = 2.736359839425547
MSE = 2.735827166213166
MSE = 2.735680667840283
MSE = 2.735333794896654
MSE = 2.7352415555868035
MSE = 2.7350928109287445
MSE = 2.7349582003929367
MSE = 2.7345255523961556
MSE = 2.73455645196

MSE = 5.287433813783579
MSE = 5.168004738206718
MSE = 10.93889046543283
MSE = 5.12504395452012
MSE = 4.662846272556348
MSE = 4.660126663796221
MSE = 4.649920755781823
MSE = 4.6265984295625255
MSE = 4.62606871187294
MSE = 4.626015533302456
Lambda = 1000, validation set MSE = 4.611524141933288
MSE = 5.287433813783579
MSE = 5.168004738206718
MSE = 5.206294429472486
MSE = 5.170586829863105
MSE = 5.188520538811423
MSE = 5.188375853557571
MSE = 5.1883228039169955
Lambda = 10000, validation set MSE = 5.1553276186901815
Optimizer done...


In [118]:
x, f, d = scipy.optimize.fmin_l_bfgs_b(cost, [playPredictionMean] + [0.0]*(numUsers + numGames), derivative,\
                                       args = (labels, 10), iprint=100, epsilon=1e-4)
unpack(x)
predicted_y = [prediction(d['userID'], d['gameID']) for d in validation_dataset]
actual_y = [d['hours_transformed'] for d in validation_dataset]
mse_new = MSE(predicted_y, actual_y)
print("After performing gradient descent using the new model, MSE on validation dataset is {}".format(mse_new))

MSE = 5.287433813783579
MSE = 5.168004738206718
MSE = 4.757567663470286
MSE = 15.507782387404413
MSE = 4.569174391701321
MSE = 4.0776778955773665
MSE = 4.064313994357759
MSE = 4.012709273721392
MSE = 3.8359045390477338
MSE = 3.4836266355862926
MSE = 3.3055999294150458
MSE = 3.0896945742370847
MSE = 3.0046198767136683
MSE = 2.937657914835682
MSE = 2.882526196081847
MSE = 2.85447012505486
MSE = 2.8500581581779376
MSE = 2.846394751960354
MSE = 2.838107206558664
MSE = 3.2671472187655866
MSE = 2.8380011318937135
MSE = 2.831411805684693
MSE = 2.8293254228982287
MSE = 2.8276211111662546
MSE = 2.826367050536518
MSE = 2.8248017036811532
MSE = 2.8243163529572466
MSE = 2.824360506070955
MSE = 2.8243931831085645
MSE = 2.824026849589087
MSE = 2.824069202460027
MSE = 2.824031390021091
MSE = 2.824016334913257
MSE = 2.8240038763090767
MSE = 2.823993042900665
MSE = 2.8239902169776907
MSE = 2.824016994023163
MSE = 2.8239903323640907
MSE = 2.8240068149661126
MSE = 2.8240016893081
MSE = 2.8239739357413938

In [119]:
unpack(x)
predictions = open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\predictions_Hours_gammav3.txt", 'w')
for l in open("C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\pairs_Hours.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,g = l.strip().split('-')
    pred = prediction(u, g)
    predictions.write(u + '-' + g + ',' + str(pred) + '\n')

predictions.close()